In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing 
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, ParameterGrid
from sklearn.model_selection import train_test_split
from itertools import product, chain
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

In [ ]:
data= pd.read_csv("data.csv")

In [3]:
data.head(10)

,shot_id_number,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,...,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min_1,power_of_shot_1,knockout_match_1,remaining_sec_1,distance_of_shot_1,is_goal
0,1,10.0,167.0,72.0,10.0,1.0,0.0,2000-01,27.0,38.0,...,shot - 30,NaN,20000012,1610612747,10.00,1.0,50.608,54.2000,38.0,NaN
1,2,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,...,shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0,0.0
2,3,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,...,shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4,1.0
3,4,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,...,NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0,0.0
4,5,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,...,NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0,1.0
5,6,244.0,-145.0,-11.0,9.0,3.0,0.0,NaN,32.0,34.0,...,shot - 17,NaN,20000012,1610612747,9.00,3.0,0.000,NaN,34.0,0.0
6,7,251.0,0.0,0.0,8.0,NaN,0.0,2000-01,52.0,20.0,...,NaN,shot - 4,20000012,1610612747,8.00,3.0,0.000,112.2000,89.4,1.0
7,8,254.0,1.0,28.0,8.0,3.0,0.0,2000-01,5.0,22.0,...,NaN,shot - 3,20000012,1610612747,68.64,3.0,0.000,5.0000,22.0,NaN
8,9,265.0,-65.0,NaN,6.0,3.0,0.0,2000-01,12.0,32.0,...,shot - 36,NaN,20000012,1610612747,6.00,3.0,0.000,12.0000,32.0,1.0
9,10,294.0,-33.0,NaN,3.0,3.0,0.0,2000-01,36.0,32.0,...,shot - 44,NaN,20000012,1610612747,3.00,3.0,0.000,52.2000,NaN,0.0


In [4]:
data.columns

Index(['shot_id_number', 'match_event_id', 'location_x', 'location_y',
       'remaining_min', 'power_of_shot', 'knockout_match', 'game_season',
       'remaining_sec', 'distance_of_shot', 'area_of_shot', 'shot_basics',
       'range_of_shot', 'team_name', 'date_of_game', 'home/away', 'flg',
       'Against', 'flag_home', 'lat/lng', 'type_of_shot',
       'type_of_combined_shot', 'match_id', 'team_id', 'remaining_min_1',
       'power_of_shot_1', 'knockout_match_1', 'remaining_sec_1',
       'distance_of_shot_1', 'is_goal'],
      dtype='object')

In [5]:
test=data.loc[data['is_goal'].isnull()]

In [6]:
test.head(10)

,shot_id_number,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,...,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min_1,power_of_shot_1,knockout_match_1,remaining_sec_1,distance_of_shot_1,is_goal
0,1,10.0,167.0,72.0,10.0,1.0,0.0,2000-01,27.0,38.0,...,shot - 30,NaN,20000012,1610612747,10.00,1.00,50.608,54.2,38.0,NaN
7,8,254.0,1.0,28.0,8.0,3.0,0.0,2000-01,5.0,22.0,...,NaN,shot - 3,20000012,1610612747,68.64,3.00,0.000,5.0,22.0,NaN
16,17,100.0,0.0,0.0,0.0,1.0,0.0,2000-01,1.0,20.0,...,NaN,shot - 4,20000019,1610612747,0.00,1.00,0.000,1.0,20.0,NaN
19,20,249.0,0.0,0.0,10.0,3.0,0.0,NaN,46.0,20.0,...,shot - 4,NaN,20000019,1610612747,10.00,70.36,0.000,46.0,20.0,NaN
21,22,265.0,134.0,127.0,9.0,3.0,0.0,NaN,4.0,38.0,...,shot - 24,NaN,20000019,1610612747,9.00,43.36,0.000,4.0,38.0,NaN
32,33,4.0,163.0,76.0,11.0,1.0,0.0,2000-01,26.0,37.0,...,shot - 34,NaN,20000047,1610612747,11.00,1.00,0.000,26.0,37.0,NaN
33,34,8.0,70.0,194.0,10.0,1.0,0.0,2000-01,58.0,40.0,...,NaN,shot - 3,20000047,1610612747,10.00,1.00,121.608,58.0,40.0,NaN
34,35,26.0,1.0,19.0,7.0,1.0,0.0,2000-01,33.0,21.0,...,shot - 39,NaN,20000047,1610612747,NaN,1.00,0.000,33.0,21.0,NaN
35,36,37.0,-12.0,NaN,5.0,1.0,0.0,2000-01,58.0,21.0,...,shot - 39,NaN,20000047,1610612747,5.00,104.36,102.608,58.0,84.4,NaN
36,37,53.0,1.0,4.0,4.0,1.0,0.0,2000-01,9.0,20.0,...,NaN,shot - 4,20000047,1610612747,108.64,1.00,0.000,9.0,59.4,NaN


In [7]:
train=data.loc[data['is_goal'].notnull()]

In [8]:
train.head(10)

,shot_id_number,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,...,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min_1,power_of_shot_1,knockout_match_1,remaining_sec_1,distance_of_shot_1,is_goal
1,2,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,...,shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0,0.0
2,3,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,...,shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4,1.0
3,4,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,...,NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0,0.0
4,5,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,...,NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0,1.0
5,6,244.0,-145.0,-11.0,9.0,3.0,0.0,NaN,32.0,34.0,...,shot - 17,NaN,20000012,1610612747,9.00,3.0,0.000,NaN,34.0,0.0
6,7,251.0,0.0,0.0,8.0,NaN,0.0,2000-01,52.0,20.0,...,NaN,shot - 4,20000012,1610612747,8.00,3.0,0.000,112.2000,89.4,1.0
8,9,265.0,-65.0,NaN,6.0,3.0,0.0,2000-01,12.0,32.0,...,shot - 36,NaN,20000012,1610612747,6.00,3.0,0.000,12.0000,32.0,1.0
9,10,294.0,-33.0,NaN,3.0,3.0,0.0,2000-01,36.0,32.0,...,shot - 44,NaN,20000012,1610612747,3.00,3.0,0.000,52.2000,NaN,0.0
10,11,309.0,-94.0,238.0,1.0,3.0,0.0,2000-01,56.0,45.0,...,shot - 7,NaN,20000012,1610612747,NaN,3.0,80.928,56.0000,45.0,0.0
11,12,4.0,121.0,127.0,11.0,1.0,0.0,2000-01,0.0,37.0,...,NaN,shot - 3,20000019,1610612747,11.00,1.0,106.608,64.7856,16.4,1.0


In [9]:
print(train.shape)
print(test.shape)

(24429, 30)
(6268, 30)


In [10]:
train.isnull().sum()

shot_id_number               0
match_event_id            1240
location_x                1156
location_y                1245
remaining_min             1244
power_of_shot             1200
knockout_match            1212
game_season               4632
remaining_sec             1250
distance_of_shot          1257
area_of_shot              1182
shot_basics               1283
range_of_shot             1248
team_name                 1212
date_of_game              1237
home/away                 1214
flg                       1214
Against                   1214
flag_home                    0
lat/lng                   1247
type_of_shot             12136
type_of_combined_shot    12293
match_id                     0
team_id                      0
remaining_min_1           1209
power_of_shot_1           1208
knockout_match_1          1214
remaining_sec_1           1200
distance_of_shot_1        1244
is_goal                      0
dtype: int64

In [11]:
test.isnull().sum()

shot_id_number              0
match_event_id            323
location_x                305
location_y                295
remaining_min             318
power_of_shot             286
knockout_match            305
game_season              1230
remaining_sec             344
distance_of_shot          310
area_of_shot              320
shot_basics               292
range_of_shot             316
team_name                 323
date_of_game              313
home/away                 283
flg                       283
Against                   283
flag_home                   0
lat/lng                   318
type_of_shot             3144
type_of_combined_shot    3124
match_id                    0
team_id                     0
remaining_min_1           326
power_of_shot_1           331
knockout_match_1          279
remaining_sec_1           339
distance_of_shot_1        324
is_goal                  6268
dtype: int64

In [328]:
df=data.drop(['is_goal'],axis=1)

In [329]:
df.columns

Index(['shot_id_number', 'match_event_id', 'location_x', 'location_y',
       'remaining_min', 'power_of_shot', 'knockout_match', 'game_season',
       'remaining_sec', 'distance_of_shot', 'area_of_shot', 'shot_basics',
       'range_of_shot', 'team_name', 'date_of_game', 'home/away', 'flg',
       'Against', 'flag_home', 'lat/lng', 'type_of_shot',
       'type_of_combined_shot', 'match_id', 'team_id', 'remaining_min_1',
       'power_of_shot_1', 'knockout_match_1', 'remaining_sec_1',
       'distance_of_shot_1'],
      dtype='object')

In [330]:
df['power_of_shot'] = np.where((df.power_of_shot.isnull()) & (df.power_of_shot_1 <= 10), df.power_of_shot_1, df.power_of_shot)
df['knockout_match'] = np.where((df.knockout_match.isnull()) & (df.knockout_match_1 < 2), df.knockout_match_1, df.knockout_match)
df['remaining_min'] = np.where((df.remaining_min.isnull()) & (df.remaining_min_1 < 90), df.remaining_min_1, df.remaining_min)
df['remaining_sec'] = np.where((df.remaining_sec.isnull()) & (df.remaining_sec_1 <= 60), df.remaining_sec_1, df.remaining_sec)
df['distance_of_shot'] = np.where((df.distance_of_shot.isnull()), df.distance_of_shot_1, df.distance_of_shot)

In [331]:
df.isnull().sum()

shot_id_number               0
match_event_id            1563
location_x                1461
location_y                1540
remaining_min              198
power_of_shot              357
knockout_match             375
game_season               5862
remaining_sec              330
distance_of_shot            97
area_of_shot              1502
shot_basics               1575
range_of_shot             1564
team_name                 1535
date_of_game              1550
home/away                 1497
flg                       1497
Against                   1497
flag_home                    0
lat/lng                   1565
type_of_shot             15280
type_of_combined_shot    15417
match_id                     0
team_id                      0
remaining_min_1           1535
power_of_shot_1           1539
knockout_match_1          1493
remaining_sec_1           1539
distance_of_shot_1        1568
dtype: int64

In [332]:
df=df.drop(['team_id'],axis=1)

In [333]:
df['remaining_min']=df['remaining_min'].fillna(0)
df['remaining_sec']=df['remaining_sec'].fillna(0)
df['remaining_time']= df['remaining_min']*60+df['remaining_sec']
df=df.drop(['remaining_min','remaining_sec'],axis=1)

In [334]:
df['Against']=df['Against'].fillna(method='ffill')
df['flg']=df['flg'].fillna(method='ffill')
df['game_season']=df['game_season'].fillna(method='ffill')
df['date_of_game']=df['date_of_game'].fillna(method='ffill')
df['knockout_match']= df['knockout_match'].fillna(method='ffill')
df['power_of_shot']= df['power_of_shot'].fillna(method='ffill')

In [335]:
df['flag_home'] = np.where(df['flg']=='@', 1, 0)

In [336]:
df=df.drop(['flg','home/away'],axis=1)

In [337]:
cols=['remaining_min_1','remaining_sec_1','power_of_shot_1','knockout_match_1','distance_of_shot_1','match_event_id','team_name']
df=df.drop(cols,axis=1)

In [338]:
df=df.sort_values(by=['date_of_game','remaining_time'])

In [339]:
df['area_of_shot']=df['area_of_shot'].fillna('N')
df['range_of_shot']=df['range_of_shot'].fillna('N')
df['shot_basics']=df['shot_basics'].fillna('N')

In [340]:
for i in range(df.shape[0]):
    if(df['area_of_shot'].iloc[i]=='N' and ((df['shot_basics'].iloc[i-1]==df['shot_basics'].iloc[i]) or (df['range_of_shot'].iloc[i-1]==df['range_of_shot'].iloc[i]))):
           df['area_of_shot'].iloc[i]= df['area_of_shot'].iloc[i-1]
    elif(df['area_of_shot'].iloc[i]=='N' and ((df['shot_basics'].iloc[i+1]==df['shot_basics'].iloc[i]) or (df['range_of_shot'].iloc[i+1]==df['range_of_shot'].iloc[i]))):
           df['area_of_shot'].iloc[i]= df['area_of_shot'].iloc[i+1]
    if(df['range_of_shot'].iloc[i]=='N' and ((df['shot_basics'].iloc[i-1]==df['shot_basics'].iloc[i]) or (df['area_of_shot'].iloc[i-1]==df['area_of_shot'].iloc[i]))):
           df['range_of_shot'].iloc[i]= df['range_of_shot'].iloc[i-1]
    elif(df['range_of_shot'].iloc[i]=='N' and ((df['shot_basics'].iloc[i+1]==df['shot_basics'].iloc[i]) or (df['area_of_shot'].iloc[i+1]==df['area_of_shot'].iloc[i]))):
           df['range_of_shot'].iloc[i]= df['range_of_shot'].iloc[i+1]
    if(df['shot_basics'].iloc[i]=='N' and ((df['range_of_shot'].iloc[i-1]==df['range_of_shot'].iloc[i]) or (df['area_of_shot'].iloc[i-1]==df['area_of_shot'].iloc[i]))):
           df['shot_basics'].iloc[i]= df['shot_basics'].iloc[i-1]
    elif(df['shot_basics'].iloc[i]=='N' and ((df['range_of_shot'].iloc[i+1]==df['range_of_shot'].iloc[i]) or (df['area_of_shot'].iloc[i+1]==df['area_of_shot'].iloc[i]))):
           df['shot_basics'].iloc[i]= df['shot_basics'].iloc[i+1]

C:\Users\jyoti.prakash\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [341]:
df['area_of_shot'] = df['area_of_shot'].replace('N', np.nan)
df['range_of_shot'] = df['range_of_shot'].replace('N', np.nan)
df['shot_basics'] = df['shot_basics'].replace('N', np.nan)

In [342]:
df['area_of_shot']= df['area_of_shot'].fillna(method='ffill')
df['range_of_shot']= df['range_of_shot'].fillna(method='ffill')
df['shot_basics']= df['shot_basics'].fillna(method='ffill')

In [468]:
cols=['location_x','location_y','lat/lng','type_of_shot','type_of_combined_shot']
df_new= df.drop(cols,axis=1)

In [469]:
df_new.isnull().sum()

shot_id_number       0
power_of_shot        0
knockout_match       0
game_season          0
distance_of_shot    97
area_of_shot         0
shot_basics          0
range_of_shot        0
date_of_game         0
Against              0
flag_home            0
match_id             0
remaining_time       0
dtype: int64

In [470]:
df_new=df_new.sort_values(by=['shot_id_number'])

In [471]:
df_new.dtypes

shot_id_number        int64
power_of_shot       float64
knockout_match      float64
game_season          object
distance_of_shot    float64
area_of_shot         object
shot_basics          object
range_of_shot        object
date_of_game         object
Against              object
flag_home             int32
match_id              int64
remaining_time      float64
dtype: object

In [472]:
df_new['power_of_shot']= df_new['power_of_shot'].astype(object)
df_new['knockout_match']= df_new['knockout_match'].astype(object)
df_new['distance_of_shot']= df_new['distance_of_shot'].astype(str).astype(float)
df_new['area_of_shot']= df_new['area_of_shot'].astype(object)
df_new['shot_basics']= df_new['shot_basics'].astype(object)
df_new['range_of_shot']= df_new['range_of_shot'].astype(object)
df_new['Against']= df_new['Against'].astype(object)
df_new['flag_home']= df_new['flag_home'].astype(object)

In [473]:
df_new['distance_of_shot']= df_new['distance_of_shot'].fillna(df_new['distance_of_shot'].mean())

In [474]:
temp=df_new.groupby(['match_id'])[['shot_id_number']].count()

In [475]:
df_new= pd.merge(df_new,temp,on='match_id',how='left')

In [476]:
df_new.rename(columns={'shot_id_number_y':'no_of_attempts','shot_id_number_x':'shot_id_number'}, inplace=True)
df_new=df_new.sort_values(by=['shot_id_number'])

In [477]:
df_new['date_of_game']=pd.to_datetime(df_new['date_of_game'],format="%M/%d/%Y")

In [478]:
df_new['flag1']= np.where(df_new['area_of_shot']=='Center(C)',1,0)
df_new['flag2']= np.where((df_new['Against']=='BKN') & (df_new['flag_home']==1),1,0)
df_new['flag3']= np.where((df_new['Against']=='NOP') & (df_new['flag_home']==1),1,0)
df_new['flag4']= np.where((df_new['Against']=='SAN') & (df_new['flag_home']==0),1,0)
df_new['flag5']= np.where((df_new['Against']=='UTH') & (df_new['flag_home']==1),1,0)

In [479]:
df_new.head(200)

,shot_id_number,power_of_shot,knockout_match,game_season,distance_of_shot,area_of_shot,shot_basics,range_of_shot,date_of_game,Against,flag_home,match_id,remaining_time,no_of_attempts,flag1,flag2,flag3,flag4,flag5
0,1,1,0,2000-01,38.0,Right Side(R),Mid Range,16-24 ft.,2000-01-31 00:10:00,POR,1,20000012,627.0,11,0,0,0,0,0
1,2,1,0,2000-01,35.0,Left Side(L),Mid Range,8-16 ft.,2000-01-31 00:10:00,POR,1,20000012,622.0,11,0,0,0,0,0
2,3,1,0,2000-01,36.0,Left Side Center(LC),Mid Range,16-24 ft.,2000-01-31 00:10:00,POR,1,20000012,465.0,11,0,0,0,0,0
3,4,1,0,2000-01,42.0,Right Side Center(RC),Mid Range,16-24 ft.,2000-01-31 00:10:00,POR,1,20000012,412.0,11,0,0,0,0,0
4,5,2,0,2000-01,20.0,Center(C),Goal Area,Less Than 8 ft.,2000-01-31 00:10:00,POR,1,20000012,2577.4,11,1,0,0,0,0
5,6,3,0,2000-01,34.0,Left Side(L),Mid Range,8-16 ft.,2000-01-31 00:10:00,POR,1,20000012,572.0,11,0,0,0,0,0
6,7,3,0,2000-01,20.0,Center(C),Goal Area,Less Than 8 ft.,2000-01-31 00:10:00,POR,1,20000012,532.0,11,1,0,0,0,0
7,8,3,0,2000-01,22.0,Center(C),Goal Area,Less Than 8 ft.,2000-01-31 00:10:00,POR,1,20000012,485.0,11,1,0,0,0,0
8,9,3,0,2000-01,32.0,Left Side(L),Goal Line,8-16 ft.,2000-01-31 00:10:00,POR,1,20000012,372.0,11,0,0,0,0,0
9,10,3,0,2000-01,32.0,Center(C),Goal Line,8-16 ft.,2000-01-31 00:10:00,POR,1,20000012,216.0,11,1,0,0,0,0


In [480]:
#le = preprocessing.LabelEncoder()

In [481]:
df_new1= df_new

In [482]:
#df_new['area_of_shot']=le.fit_transform(df_new['area_of_shot'].get_values())
#df_new['shot_basics']=le.fit_transform(df_new['shot_basics'].get_values())
#df_new['range_of_shot']=le.fit_transform(df_new['range_of_shot'].get_values())
#df_new['Against']=le.fit_transform(df_new['Against'].get_values())

In [483]:
df_new.dtypes

shot_id_number               int64
power_of_shot               object
knockout_match              object
game_season                 object
distance_of_shot           float64
area_of_shot                object
shot_basics                 object
range_of_shot               object
date_of_game        datetime64[ns]
Against                     object
flag_home                   object
match_id                     int64
remaining_time             float64
no_of_attempts               int64
flag1                        int32
flag2                        int32
flag3                        int32
flag4                        int32
flag5                        int32
dtype: object

In [484]:
cat_cols= ['area_of_shot','shot_basics','range_of_shot','Against','flag1','flag2','flag3','flag4','flag5']
df_new[cat_cols]= df_new[cat_cols].astype(object)

In [485]:
cols= ['match_id','date_of_game']
df_new=df_new.drop(cols,axis=1)

In [486]:
df_new.shape

(30697, 17)

In [487]:
df_new=df_new.sort_values(by=['shot_id_number'])
data=data.sort_values(by=['shot_id_number'])

In [488]:
df_new['is_goal']=data['is_goal']

In [489]:
train=df_new.loc[df_new['is_goal'].notnull()]
test=df_new.loc[df_new['is_goal'].isnull()]

In [490]:
RANDOM_STATE=0

In [491]:
def cross_val(X, y, X_test, param, cat_features, n_splits=3):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    
    acc = []
    predict = None
    
    for tr_ind, val_ind in skf.split(X, y):
        X_train = X[tr_ind]
        y_train = y[tr_ind]
        
        X_valid = X[val_ind]
        y_valid = y[val_ind]
        
        clf = CatBoostClassifier(iterations=500,
                                loss_function = param['loss_function'],
                                depth=param['depth'],
                                l2_leaf_reg = param['l2_leaf_reg'],
                                eval_metric = 'Accuracy',
                                leaf_estimation_iterations = 10,
                                use_best_model=True,
                                logging_level='Silent'
        )
        
        clf.fit(X_train, 
                y_train,
                cat_features=cat_features,
                eval_set=(X_valid, y_valid)
        )
        
        y_pred = clf.predict(X_valid)
        accuracy = accuracy_score(y_valid, y_pred)
        acc.append(accuracy)
    return sum(acc)/n_splits

In [492]:
def catboost_GridSearchCV(X, y, X_test, params, cat_features, n_splits=5):
    ps = {'acc':0,
          'param': []
    }
    
    predict=None
    
    for prms in tqdm(list(ParameterGrid(params)), ascii=True, desc='Params Tuning:'):
                          
        acc = cross_val(X, y, X_test, prms, cat_features, n_splits=5)

        if acc>ps['acc']:
            ps['acc'] = acc
            ps['param'] = prms
    print('Acc: '+str(ps['acc']))
    print('Params: '+str(ps['param']))
    
    return ps['param']

In [493]:
columns=df_new.columns
cat_features = np.where(df_new[columns].dtypes== np.object)[0]

In [494]:
columns[cat_features]

Index(['power_of_shot', 'knockout_match', 'game_season', 'area_of_shot',
       'shot_basics', 'range_of_shot', 'Against', 'flag_home', 'flag1',
       'flag2', 'flag3', 'flag4', 'flag5'],
      dtype='object')

In [495]:
y_train=train['is_goal']

In [496]:
X_train= train.drop(['is_goal'],axis=1)
X_test= test.drop(['is_goal'],axis=1)

In [497]:
 params = {'depth':[2, 3, 4],
              'loss_function': ['Logloss', 'CrossEntropy'],
              'l2_leaf_reg':np.logspace(-20, -19, 3)
    }
y_train=y_train.astype(object)
y_train.unique()

array([0.0, 1.0], dtype=object)

In [498]:
y_train = le.fit_transform(y_train)
from sklearn.utils.multiclass import type_of_target
type_of_target(y_train)

'binary'

In [499]:
X_train=X_train.values
X_test=X_test.values

param = catboost_GridSearchCV(X_train, y_train, X_test, params, cat_features)

Params Tuning:: 100%|#################################################################| 18/18 [47:08<00:00, 156.49s/it]


Acc: 0.6101353144425763
Params: {'depth': 2, 'l2_leaf_reg': 1e-20, 'loss_function': 'Logloss'}


In [500]:
clf = CatBoostClassifier(iterations=2500,
                            loss_function = param['loss_function'],
                            depth=param['depth'],
                            l2_leaf_reg = param['l2_leaf_reg'],
                            eval_metric = 'Accuracy',
                            leaf_estimation_iterations = 10,
                            use_best_model=True
)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                        y_train, 
                                                        shuffle=True,
                                                        random_state=RANDOM_STATE,
                                                        train_size=0.8,
                                                        stratify=y_train)

C:\Users\jyoti.prakash\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [501]:
clf.fit(X_train, 
            y_train,
            cat_features=cat_features,
            logging_level='Silent',
            eval_set=(X_valid, y_valid)
)

In [543]:
y_pred=clf.predict(X_valid)

In [544]:
mean_absolute_error(y_valid, y_pred)

0.3780188293082276

In [537]:
sub = np.around(clf.predict_proba(X_test)[:,0],1)
sub

array([0.5, 0.4, 0.5, ..., 0.6, 0.6, 0.4])

In [538]:
result = pd.DataFrame({'shot_id_number': test['shot_id_number'],'is_goal':sub})

In [539]:
sample= pd.read_csv("sample_submission.csv")

In [540]:
final=pd.merge(sample,result,on='shot_id_number',how='left')
final.rename(columns={'is_goal_y':'is_goal'}, inplace=True)
final=final.drop(['is_goal_x'],axis=1)

In [541]:
final.to_csv("submission1.csv",index=False)